In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import plotly.express as px
import millify as mil
# import warnings
# warnings.filters=["ignore"]

In [3]:
data = pd.read_parquet("DKHousingPrices.parquet")

In [4]:
data.head(5)

,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%,dk_ann_infl_rate%,yield_on_mortgage_credit_bonds%
0,2024-10-26,2024Q4,0,Villa,regular_sale,1974,4350000,0.0,5,215.0,20232.558594,Kildevangen 5,8382,Hinnerup,East & mid jutland,Jutland,3.1,NaN,NaN
1,2024-10-26,2024Q4,2,Summerhouse,regular_sale,1956,450000,0.0,3,36.0,12500.000000,Lykkestien 2,4400,Kalundborg,Other islands,Zealand,3.1,NaN,NaN
2,2024-10-26,2024Q4,1,Farm,regular_sale,1955,6600000,0.0,3,180.0,36666.667969,Sæderupvej 58,9260,Gistrup,North jutland,Jutland,3.1,NaN,NaN
3,2024-10-25,2024Q4,3,Apartment,family_sale,1945,1495000,0.0,2,64.0,23359.375000,"Tage-Hansens Gade 5, 1. tv",8000,Aarhus C,East & mid jutland,Jutland,3.1,NaN,NaN
4,2024-10-25,2024Q4,4,Villa,regular_sale,1967,3375000,0.0,5,176.0,19176.136719,Chr.Winthers Vej 5,8600,Silkeborg,East & mid jutland,Jutland,3.1,NaN,NaN


In [5]:
data.duplicated().sum()

0

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507908 entries, 0 to 1507907
Data columns (total 19 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   date                                 1507908 non-null  datetime64[ns]
 1   quarter                              1507908 non-null  period[Q-DEC] 
 2   house_id                             1507908 non-null  int64         
 3   house_type                           1507908 non-null  category      
 4   sales_type                           1507908 non-null  category      
 5   year_build                           1507908 non-null  int16         
 6   purchase_price                       1507908 non-null  int32         
 7   %_change_between_offer_and_purchase  1507908 non-null  float32       
 8   no_rooms                             1507908 non-null  int8          
 9   sqm                                  1507906 non-null  fl

In [7]:
data[data["sqm"].isna()]

,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%,dk_ann_infl_rate%,yield_on_mortgage_credit_bonds%
212845,2022-02-23,2022Q1,201482,Villa,regular_sale,1920,1995000,0.0,2,NaN,NaN,Grundtvigsvej 17,4500,Nykøbing Sj,Other islands,Zealand,0.00,7.70,3.73
1086493,2006-08-21,2006Q3,1082137,Summerhouse,regular_sale,1971,995000,0.0,2,NaN,NaN,Fjordengen 12,4300,Holbæk,Other islands,Zealand,2.75,1.92,5.14


In [8]:
#set a query for villas on the same area, same purchase_price range and same room num to get the mis vals
zealand_villa = data[(data["region"]== "Zealand") & (data["house_type"]=="Villa") & ( data["purchase_price"] < 2000000) & (data["purchase_price"] > 1900000 ) & (data["no_rooms"] == 2)]
zealand_villa[["sqm" , "sqm_price"]].describe().round(3)
# so i will fillna with median value for sqm 85.000 and sqm_price 23077.189

,sqm,sqm_price
count,66.000,66.000
mean,88.515,24960.367
std,31.492,9085.290
min,39.000,9695.432
25%,72.000,19211.340
50%,85.000,23077.189
75%,100.250,26754.166
max,197.000,50641.027


In [9]:
#set a query for Summerhouse on the same area, same purchase_price range and same room num to get the mis vals
zealand_villa = data[(data["region"]== "Zealand") & (data["house_type"]=="Summerhouse") & ( data["purchase_price"] < 1000000) & (data["purchase_price"] > 990000 ) & (data["no_rooms"] == 2)]
zealand_villa[["sqm" , "sqm_price"]].describe().round(3)
# so i will fillna with median value for sqm 50.000 and sqm_price 19900.000

,sqm,sqm_price
count,69.000,69.000
mean,51.536,20657.084
std,13.747,5436.063
min,26.000,11306.818
25%,40.000,17456.141
50%,50.000,19900.000
75%,57.000,24875.000
max,88.000,38269.230


In [10]:
data.loc[212845,["sqm" , "sqm_price"]] = 85 , 23077.189
data.loc[1086493,["sqm" , "sqm_price"]] = 50.000 , 19900.000

C:\Users\mokam\AppData\Local\Temp\ipykernel_20780\2500973225.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '23077.189' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[212845,["sqm" , "sqm_price"]] = 85 , 23077.189


In [11]:
data.iloc[1086493]

date                                   2006-08-21 00:00:00
quarter                                             2006Q3
house_id                                           1082137
house_type                                     Summerhouse
sales_type                                    regular_sale
year_build                                            1971
purchase_price                                      995000
%_change_between_offer_and_purchase                    0.0
no_rooms                                                 2
sqm                                                   50.0
sqm_price                                          19900.0
address                                      Fjordengen 12
zip_code                                              4300
city                                                Holbæk
area                                         Other islands
region                                             Zealand
nom_interest_rate%                                    2.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507908 entries, 0 to 1507907
Data columns (total 19 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   date                                 1507908 non-null  datetime64[ns]
 1   quarter                              1507908 non-null  period[Q-DEC] 
 2   house_id                             1507908 non-null  int64         
 3   house_type                           1507908 non-null  category      
 4   sales_type                           1507908 non-null  category      
 5   year_build                           1507908 non-null  int16         
 6   purchase_price                       1507908 non-null  int32         
 7   %_change_between_offer_and_purchase  1507908 non-null  float32       
 8   no_rooms                             1507908 non-null  int8          
 9   sqm                                  1507908 non-null  fl

In [13]:
# remove dk_ann_infl_rate% & yield_on_mortgage_credit_bonds% columns from data
data.drop( columns= ["dk_ann_infl_rate%","yield_on_mortgage_credit_bonds%"], inplace=True )

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507908 entries, 0 to 1507907
Data columns (total 17 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   date                                 1507908 non-null  datetime64[ns]
 1   quarter                              1507908 non-null  period[Q-DEC] 
 2   house_id                             1507908 non-null  int64         
 3   house_type                           1507908 non-null  category      
 4   sales_type                           1507908 non-null  category      
 5   year_build                           1507908 non-null  int16         
 6   purchase_price                       1507908 non-null  int32         
 7   %_change_between_offer_and_purchase  1507908 non-null  float32       
 8   no_rooms                             1507908 non-null  int8          
 9   sqm                                  1507908 non-null  fl

In [15]:
data.sample(8)

,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%
878804,2012-03-27,2012Q1,872334,Apartment,regular_sale,1898,360000,-9.0,2,64.0,5625.000000,"Nørre Voldgade 98, 2. tv",7000,Fredericia,South jutland,Jutland,0.75
1101900,2006-04-23,2006Q2,1097519,Villa,regular_sale,1944,510000,0.0,5,134.0,3805.970215,Roostvej 7,6535,Branderup J,South jutland,Jutland,2.50
974308,2009-07-30,2009Q3,967780,Apartment,regular_sale,1988,748000,0.0,2,60.0,12466.666992,Klinkbjerg 6A,6200,Aabenraa,South jutland,Jutland,1.20
978416,2009-06-20,2009Q2,971860,Summerhouse,regular_sale,1960,600000,-27.0,4,78.0,7692.307617,Skovvangen 22,4573,Højby,Other islands,Zealand,2.00
51808,2024-02-18,2024Q1,33731,Summerhouse,regular_sale,2005,1450000,-9.0,3,102.0,14215.686523,Torilisvej 55,4873,Væggerløse,Other islands,Zealand,3.60
1018932,2008-03-24,2008Q1,1012465,Apartment,regular_sale,2003,1234574,0.0,3,98.0,12597.694336,"Astergade 20, 3. 23",6700,Esbjerg,South jutland,Jutland,4.00
1373116,1998-04-05,1998Q2,1368777,Villa,regular_sale,1970,657160,0.0,5,120.0,5476.333496,Doktorbakken 3,9560,Hadsund,North jutland,Jutland,3.50
1408372,1996-11-19,1996Q4,1404155,Townhouse,regular_sale,1986,590132,0.0,3,88.0,6706.045410,Blåbærvej 26A,9440,Aabybro,North jutland,Jutland,3.25


In [16]:
data.sales_type.unique()

['regular_sale', 'family_sale', 'other_sale', 'auction', '-']
Categories (5, object): ['-', 'auction', 'family_sale', 'other_sale', 'regular_sale']

## New df cols will be: 
* (date, house_id, house_type, year_build, purchase_price, no_rooms, sqm , sqm_price, city, area, region )

In [17]:
df = data[["date", "house_id","region", "city", "house_type", "year_build", "no_rooms", "sqm" , "sqm_price", "purchase_price","nom_interest_rate%"]]

In [18]:
df.sample(5)

,date,house_id,region,city,house_type,year_build,no_rooms,sqm,sqm_price,purchase_price,nom_interest_rate%
214394,2022-02-17,203084,Jutland,Aalborg,Apartment,1934,2,67.0,23134.328125,1550000,0.0
719919,2015-10-05,713268,Zealand,København NV,Apartment,1974,3,82.0,26817.072266,2199000,0.0
53557,2024-02-09,35529,Zealand,Hvidovre,Apartment,1966,2,70.0,26071.427734,1825000,3.6
588669,2018-05-07,579728,Bornholm,Astermarie,Farm,1949,10,223.0,14573.991211,3250000,0.0
986170,2009-03-26,979596,Zealand,Gørlev,Villa,1965,4,122.0,4508.196777,550000,3.5


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507908 entries, 0 to 1507907
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   date                1507908 non-null  datetime64[ns]
 1   house_id            1507908 non-null  int64         
 2   region              1507908 non-null  category      
 3   city                1507908 non-null  object        
 4   house_type          1507908 non-null  category      
 5   year_build          1507908 non-null  int16         
 6   no_rooms            1507908 non-null  int8          
 7   sqm                 1507908 non-null  float32       
 8   sqm_price           1507908 non-null  float64       
 9   purchase_price      1507908 non-null  int32         
 10  nom_interest_rate%  1507908 non-null  float32       
dtypes: category(2), datetime64[ns](1), float32(2), float64(1), int16(1), int32(1), int64(1), int8(1), object(1)
memory usage: 70.5+ 

## What are the Top cites by Sales values and number of sold properties ?

In [97]:
area_city_sales = df.groupby(["region" ,"city"])["purchase_price"].agg(["count" , "sum"]).sort_values(by="sum",ascending=False).reset_index().head(11)

C:\Users\mokam\AppData\Local\Temp\ipykernel_20780\1461639738.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [104]:
area_city_sales = area_city_sales.rename(columns={"count" : "prop sold count" , "sum" : "sales amount"})


In [109]:
fig1 = px.bar(area_city_sales , x ="region" , y = "sales amount" , color= "city", barmode="group" , title= "Top 10 cities by sales value")
fig1.update_layout(title_x = 0.5)
fig2 = px.bar(area_city_sales , x ="region" , y = "prop sold count" , color= "city", barmode="group", title="Top 10 cities by sold properties number")
fig2.update_layout(title_x = 0.5)
display(fig1)
display(fig2)